## Imports

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv 
import requests
import json
import datetime
import time

## Fetching data for analysis

In [ ]:
response = requests.get('https://api.pokemontcg.io/v2/cards')

In [ ]:
# Initialize an empty list to store all cards
all_cards = []

# Loop through all pages
for page in range(1, 71):
    # Specify the page number and page size
    params = {
        'page': page,  # Page number
        'pageSize': 250  # Number of results per page
    }

    # Make the request
    response = requests.get('https://api.pokemontcg.io/v2/cards', params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Add the cards from this page to the list
        all_cards.extend(response.json()['data'])
    else:
        print(f"Failed to retrieve page {page}")

    # Wait for a short period to avoid hitting rate limits
    time.sleep(1)

# Now all_cards contains the cards from all pages

In [23]:
len(all_cards)

17500

In [25]:
#response_data = all_cards
# Only 250 cards are returned by the API. Just first page.
#cards = response_data['data']
cards = all_cards
len(cards)

17500

In [26]:
cards[0].keys()

dict_keys(['id', 'name', 'supertype', 'subtypes', 'hp', 'types', 'evolvesFrom', 'attacks', 'weaknesses', 'resistances', 'retreatCost', 'convertedRetreatCost', 'set', 'number', 'artist', 'rarity', 'flavorText', 'nationalPokedexNumbers', 'legalities', 'images', 'tcgplayer', 'cardmarket'])

In [ ]:
type_mapping = {
    "Colorless": 1,
    "Darkness": 2,
    "Dragon": 3,
    "Fairy": 4,
    "Fighting": 5,
    "Fire": 6,
    "Grass": 7,
    "Lightning": 8,
    "Metal": 9,
    "Psychic": 10,
    "Water": 11
}

In [ ]:

clean_cards = []

for card in cards:
    try: 
        clean_card_row = {}
        clean_card_row["name"] = card["name"]
        clean_card_row["hp"] = card["hp"]
        clean_card_row["average_price"] = card["cardmarket"]["prices"]["averageSellPrice"] # in dollars.
        clean_card_row["age"] = card.get("set", {}).get("releaseDate", "No release date available")
        clean_card_row["converted_retreat_cost"] = card["convertedRetreatCost"]
        clean_card_row['pokedex_nr'] = card['nationalPokedexNumbers'][0]
        clean_card_row["type"] = type_mapping.get(card["types"][0],0)
        # Calculate age of card in days
        release_date = datetime.datetime.strptime(clean_card_row["age"], "%Y/%m/%d").date()  
        # Get today's date
        today = datetime.date.today()
        clean_card_row["age"] = (today - release_date).days
        # get damage, so we take the mean of the attacks. 
        damage = 0
        converted_energy_cost = 0
        for attack in card["attacks"]:
            damage_str = ''.join(filter(str.isdigit, attack["damage"]))
            if not damage_str:
                continue
            damage += int(damage_str)
            converted_energy_cost += int(attack["convertedEnergyCost"])

        clean_card_row["damage"] = round(damage / len(card["attacks"])) # round up to whole numbers
        clean_card_row["converted_energy_cost"] = converted_energy_cost / len(card["attacks"])
        
        resistance = 0
        if "resistances" in card and card["resistances"]:
            if ("×" in card["resistances"][0]["value"]):
                filtered_string = ''.join([char for char in card["resistances"][0]["value"] if char.isdigit()])
                resulting_int = int(filtered_string)
                resistance = int(card["hp"]) * resulting_int
            if ("-" in card["resistances"][0]["value"]):
                filtered_string = ''.join([char for char in card["resistances"][0]["value"] if char.isdigit()])
                resulting_int = int(filtered_string)
                resistance = int(card["hp"]) + resulting_int
        else:
            resistance = int(card["hp"])
        clean_card_row["resistance"] = resistance

        weakness = 0
        if "weaknesses" in card and card["weaknesses"]:
            if ("×" in card["weaknesses"][0]["value"]):
                filtered_string = ''.join([char for char in card["weaknesses"][0]["value"] if char.isdigit()])
                resulting_int = int(filtered_string)
                weakness = int(card["hp"]) * resulting_int
            if ("+" in card["weaknesses"][0]["value"]):
                filtered_string = ''.join([char for char in card["weaknesses"][0]["value"] if char.isdigit()])
                resulting_int = int(filtered_string)
                weakness = int(card["hp"]) + resulting_int
            if ("-" in card["weaknesses"][0]["value"]):
                filtered_string = ''.join([char for char in card["weaknesses"][0]["value"] if char.isdigit()])
                resulting_int = int(filtered_string)
                weakness = int(card["hp"]) - resulting_int
        else:
            weakness = int(card["hp"])
        clean_card_row["weakness"] = weakness

        
    except (KeyError, IndexError, ValueError):
        #print(KeyError, IndexError, ValueError)
        continue
    
    print(clean_card_row)
    clean_cards.append(clean_card_row)

print(len(clean_cards))
df = pd.DataFrame(clean_cards)
df.to_csv('cards.csv', mode='w', index=False)
    